# Packages required

In [1]:
import pandas as pd
import requests
import json

# List of coordinates (lat, long)

In [4]:
locations = [(-18.726947081337237, -47.50173526122225), (-22.911980408452308, -43.23016653421126), (-22.983918759804688, -43.50574380537288),(40.453336531608684, -3.6883908445884326),(51.532290532072444, -0.17743750791303706),
 (35.139297944169684, -90.03769657594907),(-62.08350473870686, -58.383179276598746),(64.75527433421244, -147.35139924409958),(53.38049046812334, -2.8829346887260834),(51.51056029553284, -0.13915102935364002)]

#Collect data monthly




In [30]:
# Function to collect data from NASA Power API for a given location (latitude, longitude)
def collect_data(latitude, longitude):
    # Define the base URL for the NASA Power API
    base_url = r"https://power.larc.nasa.gov/api/temporal/monthly/point?parameters=PRECTOTCORR,PRECTOTCORR_SUM,T2M&community=RE&longitude={longitude}&latitude={latitude}&start=2018&end=2022&format=JSON"

    # Construct the API request URL with the provided latitude and longitude
    api_request_url = base_url.format(longitude=longitude, latitude=latitude)

    # Make a request to the API and get the response
    response = requests.get(url=api_request_url, verify=True, timeout=30.00)

    # Decode the JSON content from the response
    json_data = json.loads(response.content.decode('utf-8'))

    # Return the collected JSON data
    return json_data

# Create an empty list to store the collected data points
monthly_data = []

# Iterate through each location (latitude, longitude) in the 'locations' list
for latitude, longitude in locations:
    # Collect data from the NASA Power API for the current location
    json_data = collect_data(latitude, longitude)

    # Iterate through each parameter in the collected data
    for parameter, values in json_data['properties']['parameter'].items():
        # Iterate through each period and its corresponding value
        for period, value in values.items():
            # Create a dictionary for each data point
            data_point = {
                'period': period,
                'parameter': parameter,
                'coord': json_data['geometry']['coordinates'],
                'lat': latitude,
                'lon': longitude,
                'altitude': json_data['geometry']['coordinates'][2],
                'value': value
            }
            # Append the data point to the list
            monthly_data.append(data_point)

# Create a DataFrame from the collected data points
monthly_df = pd.DataFrame(monthly_data)

In [ ]:
monthly_data

In [11]:
# Print or use the DataFrame as needed 2340 / 38415
monthly_df

,period,parameter,coord,lat,lon,altitude,value
0,201801,T2M,"[-47.50173526122225, -18.726947081337237, 800.25]",-18.726947,-47.501735,800.25,24.01
1,201802,T2M,"[-47.50173526122225, -18.726947081337237, 800.25]",-18.726947,-47.501735,800.25,23.47
2,201803,T2M,"[-47.50173526122225, -18.726947081337237, 800.25]",-18.726947,-47.501735,800.25,24.01
3,201804,T2M,"[-47.50173526122225, -18.726947081337237, 800.25]",-18.726947,-47.501735,800.25,22.06
4,201805,T2M,"[-47.50173526122225, -18.726947081337237, 800.25]",-18.726947,-47.501735,800.25,20.57
...,...,...,...,...,...,...,...
775,201909,PRECTOTCORR_SUM,"[-0.13915102935364002, 51.51056029553284, 73.15]",51.510560,-0.139151,73.15,73.83
776,201910,PRECTOTCORR_SUM,"[-0.13915102935364002, 51.51056029553284, 73.15]",51.510560,-0.139151,73.15,89.65
777,201911,PRECTOTCORR_SUM,"[-0.13915102935364002, 51.51056029553284, 73.15]",51.510560,-0.139151,73.15,94.92
778,201912,PRECTOTCORR_SUM,"[-0.13915102935364002, 51.51056029553284, 73.15]",51.510560,-0.139151,73.15,100.20


In [38]:
# Pivot the DataFrame to transform the 'parameter' column into separate columns
# This operation rearranges the DataFrame, making each unique parameter a new column
monthly_pivoted = monthly_df.pivot_table(index=['period', 'lat', 'lon', 'altitude'],
                                         columns='parameter',
                                         values=['value'],
                                         aggfunc='first').reset_index()

# Flatten the MultiIndex columns to make it easier to work with
# The MultiIndex is present due to the pivot_table operation
monthly_pivoted.columns = [''.join(col).strip() for col in monthly_pivoted.columns.values]


In [39]:
monthly_pivoted

,period,lat,lon,altitude,valuePRECTOTCORR,valuePRECTOTCORR_SUM,valueT2M
0,201801,-62.083505,-58.383179,29.24,0.00,68.55,1.00
1,201801,-22.983919,-43.505744,100.37,10.55,263.67,25.62
2,201801,-22.911980,-43.230167,80.76,5.27,242.58,25.52
3,201801,-18.726947,-47.501735,800.25,5.27,179.30,24.01
4,201801,35.139298,-90.037697,87.35,5.27,89.65,1.03
...,...,...,...,...,...,...,...
645,202213,40.453337,-3.688391,779.85,1.24,451.83,15.09
646,202213,51.510560,-0.139151,73.15,1.72,627.42,11.56
647,202213,51.532291,-0.177438,73.15,1.72,627.42,11.56
648,202213,53.380490,-2.882935,58.39,2.37,863.76,10.80


In [ ]:
# exporting a csv file
monthly_pivoted.to_csv('monthly_pivoted.csv', index=False)

# Collecting daily data

In [32]:
# Function to collect daily data from NASA Power API for a given location
# This function retrieves daily weather data such as precipitation and temperature
def collect_data(latitude, longitude):
    # Define the base URL for the NASA Power API, targeting daily data
    base_url = r"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=PRECTOTCORR,T2M&community=RE&longitude={longitude}&latitude={latitude}&start=20180101&end=20221231&format=JSON"
    api_request_url = base_url.format(longitude=longitude, latitude=latitude)

    # Make a request to the API and get the response
    response = requests.get(url=api_request_url, verify=True, timeout=30.00)

    # Decode the JSON content from the response
    json_data = json.loads(response.content.decode('utf-8'))

    # Return the collected JSON data
    return json_data

# Create an empty list to store the collected daily data points
daily_data = []

# Iterate through each location (latitude, longitude) in the 'locations' list
for latitude, longitude in locations:
    # Collect daily data from the NASA Power API for the current location
    json_data = collect_data(latitude, longitude)

    # Iterate through each parameter in the collected data
    for parameter, values in json_data['properties']['parameter'].items():
        # Iterate through each period and its corresponding value
        for period, value in values.items():
            # Create a dictionary for each daily data point
            data_point = {
                'period': period,
                'parameter': parameter,
                'coord': json_data['geometry']['coordinates'],
                'lat': latitude,
                'lon': longitude,
                'altitude': json_data['geometry']['coordinates'][2],
                'value': value
            }
            # Append the daily data point to the list
            daily_data.append(data_point)

# Create a DataFrame from the collected daily data points
daily_df = pd.DataFrame(daily_data)


In [ ]:
json_data

In [34]:
daily_df

,period,parameter,coord,lat,lon,altitude,value
0,20180101,PRECTOTCORR,"[-47.50173526122225, -18.726947081337237, 800.25]",-18.726947,-47.501735,800.25,9.35
1,20180102,PRECTOTCORR,"[-47.50173526122225, -18.726947081337237, 800.25]",-18.726947,-47.501735,800.25,5.99
2,20180103,PRECTOTCORR,"[-47.50173526122225, -18.726947081337237, 800.25]",-18.726947,-47.501735,800.25,8.60
3,20180104,PRECTOTCORR,"[-47.50173526122225, -18.726947081337237, 800.25]",-18.726947,-47.501735,800.25,7.33
4,20180105,PRECTOTCORR,"[-47.50173526122225, -18.726947081337237, 800.25]",-18.726947,-47.501735,800.25,28.45
...,...,...,...,...,...,...,...
36515,20221227,T2M,"[-0.13915102935364002, 51.51056029553284, 73.15]",51.510560,-0.139151,73.15,6.57
36516,20221228,T2M,"[-0.13915102935364002, 51.51056029553284, 73.15]",51.510560,-0.139151,73.15,9.95
36517,20221229,T2M,"[-0.13915102935364002, 51.51056029553284, 73.15]",51.510560,-0.139151,73.15,7.16
36518,20221230,T2M,"[-0.13915102935364002, 51.51056029553284, 73.15]",51.510560,-0.139151,73.15,9.08


In [35]:
# Pivot the DataFrame to transform the 'parameter' column into separate columns
# This operation rearranges the DataFrame, making each unique parameter a new column
daily_pivoted = daily_df.pivot_table(index=['period', 'lat', 'lon', 'altitude'],
                                     columns='parameter',
                                     values=['value'],
                                     aggfunc='first').reset_index()

# Flatten the MultiIndex columns to make it easier to work with
# The MultiIndex is present due to the pivot_table operation
daily_pivoted.columns = [''.join(col).strip() for col in daily_pivoted.columns.values]


In [36]:
daily_pivoted.head()

,period,lat,lon,altitude,valuePRECTOTCORR,valueT2M
0,20180101,-62.083505,-58.383179,29.24,1.12,0.24
1,20180101,-22.983919,-43.505744,100.37,0.80,26.24
2,20180101,-22.911980,-43.230167,80.76,1.08,26.09
3,20180101,-18.726947,-47.501735,800.25,9.35,23.64
4,20180101,35.139298,-90.037697,87.35,0.00,-10.14


In [ ]:
# Exporting daily data to csv
daily_pivoted.to_csv('daily_pivoted.csv', index=False)

# Rolling averages

In [ ]:
# Create a copy of the pivoted DataFrame for testing purposes
rolling_calc_df = daily_pivoted.copy()

# Convert the 'period' column to datetime format with the specified format
rolling_calc_df['period'] = pd.to_datetime(rolling_calc_df['period'], format='%Y%m%d')

# Sort the DataFrame by 'period' in ascending order
rolling_calc_df = rolling_calc_df.sort_values(by='period')

# Calculate rolling sum for precipitation (PREC_mm) for the last 30 days
rolling_calc_df['prec_sum_last_30'] = rolling_calc_df.groupby(['lat', 'lon'])['valuePRECTOTCORR'].transform(lambda x: x.rolling(window=30, min_periods=1).sum())

# Calculate rolling sum for precipitation (PREC_mm) for the next 90 and 120 days
rolling_calc_df['prec_sum_next_30'] = rolling_calc_df.groupby(['lat', 'lon'])['valuePRECTOTCORR'].transform(lambda x: x.rolling(window=30, min_periods=1).sum().shift(-30))
rolling_calc_df['prec_sum_next_60'] = rolling_calc_df.groupby(['lat', 'lon'])['valuePRECTOTCORR'].transform(lambda x: x.rolling(window=60, min_periods=1).sum().shift(-60))
rolling_calc_df['prec_sum_next_90'] = rolling_calc_df.groupby(['lat', 'lon'])['valuePRECTOTCORR'].transform(lambda x: x.rolling(window=90, min_periods=1).sum().shift(-90))
rolling_calc_df['prec_sum_next_120'] = rolling_calc_df.groupby(['lat', 'lon'])['valuePRECTOTCORR'].transform(lambda x: x.rolling(window=120, min_periods=1).sum().shift(-120))

# Calculate rolling average for temperature (TEMP_2M) for the next 90 and 120 days
rolling_calc_df['temp_avg_next_30'] = rolling_calc_df.groupby(['lat', 'lon'])['valueT2M'].transform(lambda x: x.rolling(window=30, min_periods=1).mean().shift(-30))
rolling_calc_df['temp_avg_next_60'] = rolling_calc_df.groupby(['lat', 'lon'])['valueT2M'].transform(lambda x: x.rolling(window=60, min_periods=1).mean().shift(-60))
rolling_calc_df['temp_avg_next_90'] = rolling_calc_df.groupby(['lat', 'lon'])['valueT2M'].transform(lambda x: x.rolling(window=90, min_periods=1).mean().shift(-90))
rolling_calc_df['temp_avg_next_120'] = rolling_calc_df.groupby(['lat', 'lon'])['valueT2M'].transform(lambda x: x.rolling(window=120, min_periods=1).mean().shift(-120))

# Calculate monthly and annual averages for temperature (TEMP_2M)
rolling_calc_df['temp_monthly_avg'] = rolling_calc_df.groupby(['lat', 'lon', rolling_calc_df['period'].dt.to_period('M')])['valueT2M'].transform('mean')
rolling_calc_df['temp_annual_avg'] = rolling_calc_df.groupby(['lat', 'lon', rolling_calc_df['period'].dt.year])['valueT2M'].transform('mean')

# Calculate min and max temperature (TEMP_2M) for each year
rolling_calc_df['min_temp_yearly'] = rolling_calc_df.groupby(['lat', 'lon', rolling_calc_df['period'].dt.year])['valueT2M'].transform('min')
rolling_calc_df['max_temp_yearly'] = rolling_calc_df.groupby(['lat', 'lon', rolling_calc_df['period'].dt.year])['valueT2M'].transform('max')

# Calculate annual sum of precipitation (PREC_mm)
rolling_calc_df['annual_sum_precip'] = rolling_calc_df.groupby(['lat', 'lon', rolling_calc_df['period'].dt.year])['valuePRECTOTCORR'].transform('sum')


In [ ]:
rolling_calc_df

,period,lat,lon,altitude,valuePRECTOTCORR,valueT2M,prec_sum_last_30,prec_sum_next_30,prec_sum_next_60,prec_sum_next_90,prec_sum_next_120,temp_avg_next_30,temp_avg_next_60,temp_avg_next_90,temp_avg_next_120,temp_monthly_avg,temp_annual_avg,min_temp_yearly,max_temp_yearly,annual_sum_precip
0,2018-01-01,-21.053056,32.234722,349.40,0.00,30.59,0.00,20.86,385.22,449.27,510.01,28.085333,26.561333,26.120444,25.481750,28.166129,24.364027,15.41,33.78,759.28
126,2018-01-01,0.016020,34.445790,1247.86,0.04,22.19,0.04,31.84,178.89,442.41,816.52,23.196667,23.941500,23.391222,22.819667,23.164194,22.574466,19.05,26.88,2216.21
127,2018-01-01,0.210000,30.114440,1333.66,0.76,21.77,0.76,34.11,121.49,309.08,565.27,22.226333,22.872667,22.308222,21.972500,22.211613,21.512795,18.92,25.42,1359.51
128,2018-01-01,0.442780,33.216110,1138.51,0.12,23.84,0.12,45.16,145.01,347.26,641.58,24.326000,24.906833,24.117222,23.415667,24.310323,22.612356,19.01,27.70,1658.82
129,2018-01-01,0.442780,33.216111,1138.51,0.12,23.84,0.12,45.16,145.01,347.26,641.58,24.326000,24.906833,24.117222,23.415667,24.310323,22.612356,19.01,27.70,1658.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359593,2022-12-31,-14.263817,34.570300,1268.28,1.42,20.12,399.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.486452,18.487233,12.42,24.65,2678.24
359594,2022-12-31,-14.234310,28.293930,1178.66,27.60,21.62,284.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.219032,20.220247,13.06,28.23,1362.14
359595,2022-12-31,-14.216667,33.700000,1168.39,3.51,20.56,318.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.500968,19.659260,13.30,26.83,1961.78
359597,2022-12-31,-13.979333,33.645134,1168.39,3.51,20.56,318.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.500968,19.659260,13.30,26.83,1961.78


In [ ]:
weather_calculated = rolling_calc_df.copy()

In [ ]:
weather_calculated.to_csv('daily_weather.csv', index=False)

In [ ]:
df_subset = pd.read_csv('df_subset.csv')

In [ ]:
weather_calculated = pd.read_csv('daily_weather.csv')